# **Data Preparation for Udacity Data Science Nanodegree - Project Data Science Blog**

Author: Carolina Barros


### Table of Contents
1. [Introduction & Objective](#1-introduction--objective)
2. [Importing Libraries](#2-importing-libraries)
3. [Loading the Dataset](#3-loading-the-dataset)
4. [Data Cleaning & Preprocessing](#4-data-cleaning--preprocessing)

### 1. Introduction & Objective

This notebook analyzes the dataset from the 2024 Stack Overflow Developer Survey. Given the dataset's large size (over 155 MB), this notebook focuses on preprocessing the data to reduce its size and create a more manageable format. 


### 2. Importing Libraries

In [1]:
# Required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 3. Loading the Dataset

In [2]:
# Read survey results 

df_survey = pd.read_csv("../data/survey_results_public.csv")

In [ ]:
# Check first rows 
df_survey.head()

,ResponseId,MainBranch,Age,Employment,RemoteWork,Check,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,Under 18 years old,"Employed, full-time",Remote,Apples,Hobby,Primary/elementary school,Books / Physical media,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,3,I am a developer by profession,45-54 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN,NaN
3,4,I am learning to code,18-24 years old,"Student, full-time",NaN,Apples,NaN,Some college/university study without earning ...,"Other online resources (e.g., videos, blogs, f...",Stack Overflow;How-to videos;Interactive tutorial,...,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN,NaN
4,5,I am a developer by profession,18-24 years old,"Student, full-time",NaN,Apples,NaN,"Secondary school (e.g. American high school, G...","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Written Tutorial...,...,NaN,NaN,NaN,NaN,NaN,NaN,Too short,Easy,NaN,NaN


In [4]:
df_survey.shape

(65437, 114)

In [5]:
df_survey.columns

Index(['ResponseId', 'MainBranch', 'Age', 'Employment', 'RemoteWork', 'Check',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       ...
       'JobSatPoints_6', 'JobSatPoints_7', 'JobSatPoints_8', 'JobSatPoints_9',
       'JobSatPoints_10', 'JobSatPoints_11', 'SurveyLength', 'SurveyEase',
       'ConvertedCompYearly', 'JobSat'],
      dtype='object', length=114)

In [6]:
df_survey.dtypes

ResponseId               int64
MainBranch              object
Age                     object
Employment              object
RemoteWork              object
                        ...   
JobSatPoints_11        float64
SurveyLength            object
SurveyEase              object
ConvertedCompYearly    float64
JobSat                 float64
Length: 114, dtype: object

In [7]:
df_survey.isna().sum()

ResponseId                 0
MainBranch                 0
Age                        0
Employment                 0
RemoteWork             10631
                       ...  
JobSatPoints_11        35992
SurveyLength            9255
SurveyEase              9199
ConvertedCompYearly    42002
JobSat                 36311
Length: 114, dtype: int64

In [ ]:
# It seems that there are some columns with a lot of missing values values, 
# I am going to check how many columns have more that 70% of missing values and evaluate wheter if I remove then from the dataset

threshold = 0.7 # 70% of missing values
missing_ratio = df_survey.isna().mean()
print(missing_ratio)


ResponseId             0.000000
MainBranch             0.000000
Age                    0.000000
Employment             0.000000
RemoteWork             0.162462
                         ...   
JobSatPoints_11        0.550025
SurveyLength           0.141434
SurveyEase             0.140578
ConvertedCompYearly    0.641869
JobSat                 0.554900
Length: 114, dtype: float64


In [9]:
cols_to_drop = missing_ratio[missing_ratio > threshold].index

After reading the features description on the schema file, I am gonna keep the features I believe could be relevant for the objective which is analyzing which factors influence developer salaries.
Columns I believe could be relevant:
- **Main Branch:** Which of the following options best describes you today? For the purpose of this
survey, a developer is "someone who writes code" (Developer, Not primarily a Developer, Used to be a Developer, Learning the code, code as a hobby, none of these.)
- **Age:** What is your age?
- **Employement:** Employement status (full-time, part-time, freelancer, looking for work, not looking for work, full-time student, part-time student, retired, I prefer not to say)
- **Remote Work:** Remote, In-person, Hybrid.
- **Ed Level:** Education level.
- **Years Code:**  Including any education, how many years have you been coding in total?
- **Years Code Pro:**  how many years have you coded professionally (as a part of your work)?
- **DevType**: Which of the following describes your current job.
- **OrgSize**:  how many people are employed by the company or organization you currently work for? 
- **Country** : Where do you live?
- **Currency** : Which currency do you use day-to-day?
- **CompTotal**: What is your current total annual compensation (salary, bonuses, and perks, before taxes and deductions)?
- **Language**: Which programming, scripting, and markup languages have you done extensive development work in over the past year, and which do you want to work in over the next year?
- **Database**: Which database environments have you done extensive development work in over the past year, and which do you want to work in over the next year?
- **Platform**: Which cloud platforms have you done extensive development work in over the past year, and which do you want to work in over the next year?
- **WorkExp**: How many years of working experience do you have?
- **Industry**: What industry is the company you work for in?
- **JobSat**: How satisfied are you in your current professional developer role?


In [ ]:
print(list(df_survey.columns))

['ResponseId', 'MainBranch', 'Age', 'Employment', 'RemoteWork', 'Check', 'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline', 'TechDoc', 'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'BuildvsBuy', 'TechEndorse', 'Country', 'Currency', 'CompTotal', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageAdmired', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'DatabaseAdmired', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'PlatformAdmired', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith', 'WebframeAdmired', 'EmbeddedHaveWorkedWith', 'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'MiscTechAdmired', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 'ToolsTechAdmired', 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'NEWCollabToolsAdmired', 'OpSysPersonal use', 'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith',

In [12]:
# Filter relevant columns
columns_to_keep = ['ResponseId', 'Age', 'Employment', 'RemoteWork', 'EdLevel', 'YearsCodePro', 'DevType', 'OrgSize', 'Country',
  'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith',  'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'WorkExp', 'Industry', 'JobSat','ConvertedCompYearly']

df_survey_filtered = df_survey[columns_to_keep]

df_survey_filtered.head()

,ResponseId,Age,Employment,RemoteWork,EdLevel,YearsCodePro,DevType,OrgSize,Country,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith,WebframeHaveWorkedWith,WorkExp,Industry,JobSat,ConvertedCompYearly
0,1,Under 18 years old,"Employed, full-time",Remote,Primary/elementary school,NaN,NaN,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,35-44 years old,"Employed, full-time",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",17,"Developer, full-stack",NaN,United Kingdom of Great Britain and Northern I...,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,Dynamodb;MongoDB;PostgreSQL,Amazon Web Services (AWS);Heroku;Netlify,Express;Next.js;Node.js;React,17.0,NaN,NaN,NaN
2,3,45-54 years old,"Employed, full-time",Remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",27,Developer Experience,NaN,United Kingdom of Great Britain and Northern I...,C#,Firebase Realtime Database,Google Cloud,ASP.NET CORE,NaN,NaN,NaN,NaN
3,4,18-24 years old,"Student, full-time",NaN,Some college/university study without earning ...,NaN,"Developer, full-stack",NaN,Canada,C;C++;HTML/CSS;Java;JavaScript;PHP;PowerShell;...,MongoDB;MySQL;PostgreSQL;SQLite,Amazon Web Services (AWS);Fly.io;Heroku,jQuery;Next.js;Node.js;React;WordPress,NaN,NaN,NaN,NaN
4,5,18-24 years old,"Student, full-time",NaN,"Secondary school (e.g. American high school, G...",NaN,"Developer, full-stack",NaN,Norway,C++;HTML/CSS;JavaScript;Lua;Python;Rust,PostgreSQL;SQLite,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
missing_ratio = df_survey_filtered.isna().mean()
print(missing_ratio.sort_values(ascending=True))


ResponseId                0.000000
Age                       0.000000
Employment                0.000000
EdLevel                   0.071107
LanguageHaveWorkedWith    0.086984
DevType                   0.091569
Country                   0.099439
RemoteWork                0.162462
YearsCodePro              0.211302
DatabaseHaveWorkedWith    0.232025
OrgSize                   0.274417
WebframeHaveWorkedWith    0.309855
PlatformHaveWorkedWith    0.352568
WorkExp                   0.546770
JobSat                    0.554900
Industry                  0.558996
ConvertedCompYearly       0.641869
dtype: float64


In [14]:
# Remove missing values, especifically comp total because it has 48% of missing values which is going to be the target variable

df_survey_filtered = df_survey_filtered.dropna()
df_survey_filtered.shape

(10137, 17)

In [16]:
df_survey_filtered.to_csv('../data/dataset.csv', sep =',')